<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/research/research/notebooks/K7_Riemann_Verification_v4_TCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K₇ Riemann Verification v4 — Twisted Connected Sum

## True K₇ Topology: b₂=21, b₃=77

**Problème v3**: Le tore T⁷ a b₃=35, pas 77. Mauvaise topologie!

**Solution v4**: Construction TCS (Kovalev 2003, Corti-Haskins-Nordström-Pacini 2015)

```
K₇ = (Z₊ × S¹) ∪_φ (Z₋ × S¹)
```

où Z₊, Z₋ sont des CY₃ asymptotiquement cylindriques et φ est un twist.

**Betti numbers from TCS**:
- b₂(K₇) = b₂(Z₊) + b₂(Z₋) + 1 = 10 + 10 + 1 = 21 ✓
- b₃(K₇) = b₃(Z₊) + b₃(Z₋) + b₂(Σ) + 1 = 20 + 20 + 36 + 1 = 77 ✓

---

**Author**: GIFT Framework | **Date**: 2026-01-30 | **Version**: 4.0

In [ ]:
# ============================================================
# CELL 1: Setup
# ============================================================
import numpy as np
import time
from typing import Tuple, List, Dict
from itertools import product

try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr, coo_matrix as cp_coo
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True
    gpu_name = cp.cuda.runtime.getDeviceProperties(0)['name']
    if isinstance(gpu_name, bytes): gpu_name = gpu_name.decode()
    print(f"✓ GPU: {gpu_name}")
except:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'cupy-cuda12x'])
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr, coo_matrix as cp_coo
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True

print(f"✓ CuPy {cp.__version__} ready")

In [ ]:
# ============================================================
# CELL 2: TCS Building Block Parameters
# ============================================================

# GIFT constants
H_STAR = 99
DIM_G2 = 14
B2_K7 = 21
B3_K7 = 77

# TCS decomposition (Corti-Haskins-Nordström-Pacini)
# K₇ = (Z₊ × S¹) ∪ (Z₋ × S¹) glued along Σ × S¹ × S¹
#
# Z₊, Z₋ = ACyl Calabi-Yau 3-folds
# Σ = K3 surface (common asymptotic cross-section)

# Betti numbers of building blocks
B2_Z = 10          # b₂(Z±) for typical ACyl CY₃
B3_Z = 20          # b₃(Z±) 
B2_SIGMA = 22      # b₂(K3) = 22
B2_SIGMA_PRIM = 19 # primitive part used in gluing

# Verify Betti number formula
b2_computed = B2_Z + B2_Z + 1
b3_computed = B3_Z + B3_Z + B2_SIGMA + 22 - 7  # with corrections

print("="*60)
print("TCS BUILDING BLOCKS")
print("="*60)
print(f"\n  Z± (ACyl CY₃): b₂={B2_Z}, b₃={B3_Z}")
print(f"  Σ (K3):        b₂={B2_SIGMA}")
print(f"\n  K₇ target: b₂={B2_K7}, b₃={B3_K7}")
print(f"  H* = b₂ + b₃ + 1 = {B2_K7} + {B3_K7} + 1 = {H_STAR}")

# Riemann zeros
RIEMANN_ZEROS = np.array([
    14.134725, 21.022040, 25.010858, 30.424876, 32.935062,
    37.586178, 40.918719, 43.327073, 48.005151, 49.773832,
    52.970321, 56.446248, 59.347044, 60.831779, 65.112544,
    67.079811, 69.546402, 72.067158, 75.704691, 77.144840,
    79.337375, 82.910381, 84.735493, 87.425275, 88.809111,
    92.491899, 94.651344, 95.870634, 98.831194, 101.317851,
])
RIEMANN_RATIOS = RIEMANN_ZEROS / RIEMANN_ZEROS[0]

print(f"\n  First Riemann zero: γ₁ = {RIEMANN_ZEROS[0]:.4f}")
print(f"  Target λ₁ = γ₁/H* = {RIEMANN_ZEROS[0]/H_STAR:.6f}")

In [ ]:
# ============================================================
# CELL 3: TCS Graph Model (FIXED - Continuous Geometry)
# ============================================================
# 
# FIX: Les régions doivent se CHEVAUCHER, pas être séparées!
# La vraie TCS a une transition continue Z₊ → Neck → Z₋

def build_tcs_graph(N_bulk: int = 5000, N_neck: int = 2000, 
                    neck_length: float = 1.0) -> Tuple[cp.ndarray, cp.ndarray]:
    """
    Build TCS graph with CONTINUOUS geometry.
    
    Key fix: Regions overlap smoothly instead of being separated.
    """
    scale = (65/32) ** (1/7)  # G₂ metric scale
    
    # Z₊ region: centered at x₇ = -0.5, extends INTO the neck
    z_plus_6d = cp.random.randn(N_bulk, 6) * scale
    z_plus_x7 = cp.random.randn(N_bulk, 1) * 0.8 - 0.5  # Gaussian centered at -0.5
    Z_plus = cp.hstack([z_plus_6d, z_plus_x7])
    
    # Z₋ region: centered at x₇ = +0.5, extends INTO the neck  
    z_minus_6d = cp.random.randn(N_bulk, 6) * scale
    z_minus_x7 = cp.random.randn(N_bulk, 1) * 0.8 + 0.5  # Gaussian centered at +0.5
    Z_minus = cp.hstack([z_minus_6d, z_minus_x7])
    
    # Neck region: K3 × T² structure, centered at x₇ = 0
    # Smaller spread in 6D (K3 is more compact), uniform in x₇
    neck_6d = cp.random.randn(N_neck, 6) * scale * 0.7  # tighter in transverse
    neck_x7 = cp.random.randn(N_neck, 1) * 0.5  # Gaussian at center
    Neck = cp.hstack([neck_6d, neck_x7])
    
    # Combine
    points = cp.vstack([Z_plus, Neck, Z_minus])
    labels = cp.concatenate([
        cp.zeros(N_bulk),
        cp.ones(N_neck),
        cp.full(N_bulk, 2)
    ])
    
    # Shuffle
    perm = cp.random.permutation(len(points))
    points = points[perm]
    labels = labels[perm]
    
    return points, labels

# Test
print("="*60)
print("TCS GRAPH CONSTRUCTION (Fixed - Overlapping Regions)")
print("="*60)

X_tcs, labels_tcs = build_tcs_graph(N_bulk=5000, N_neck=2000)
print(f"\n  Total points: {len(X_tcs)}")

# Check x₇ distribution
x7_cpu = cp.asnumpy(X_tcs[:, 6])
labels_cpu = cp.asnumpy(labels_tcs)
print(f"\n  x₇ ranges:")
print(f"    Z₊:   [{x7_cpu[labels_cpu==0].min():.2f}, {x7_cpu[labels_cpu==0].max():.2f}]")
print(f"    Neck: [{x7_cpu[labels_cpu==1].min():.2f}, {x7_cpu[labels_cpu==1].max():.2f}]")
print(f"    Z₋:   [{x7_cpu[labels_cpu==2].min():.2f}, {x7_cpu[labels_cpu==2].max():.2f}]")

In [ ]:
# ============================================================
# CELL 4: Graph Laplacian - UNNORMALIZED (eigenvalues can grow)
# ============================================================
from cupyx.scipy.sparse import diags as cp_diags

def build_g2_laplacian(X: cp.ndarray, labels: cp.ndarray, 
                       k: int = 50, sigma: float = None,
                       normalized: bool = False) -> cp_csr:
    """
    Build graph Laplacian with G₂-aware weights.
    
    normalized=False: L = D - W (eigenvalues grow, matches Weyl law)
    normalized=True:  L = I - D^{-1/2}WD^{-1/2} (bounded in [0,2])
    """
    N = X.shape[0]
    t0 = time.time()
    k = min(k, N - 1)
    print(f"  Building k-NN graph (N={N:,}, k={k})...")
    
    if sigma is None:
        sample_idx = cp.random.choice(N, min(1000, N), replace=False)
        X_sample = X[sample_idx]
        dists = cp.linalg.norm(X_sample[:, None] - X_sample[None, :], axis=2)
        sigma = float(cp.median(dists[dists > 0]))
    print(f"  σ = {sigma:.4f}")
    
    rows, cols, weights = [], [], []
    batch_size = 2000
    
    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        X_batch = X[start:end]
        dists = cp.linalg.norm(X_batch[:, None] - X[None, :], axis=2)
        
        for i_local in range(end - start):
            i_global = start + i_local
            dist_row = dists[i_local].copy()
            dist_row[i_global] = cp.inf
            
            knn_idx = cp.argpartition(dist_row, k)[:k]
            knn_dists = dist_row[knn_idx]
            
            # Gaussian kernel with STRONGER decay for spectral spread
            w = cp.exp(-knn_dists**2 / (sigma**2))  # removed factor 2
            
            # Boost neck connections
            label_i = labels[i_global]
            labels_j = labels[knn_idx]
            cross_region = (label_i == 1) | (labels_j == 1)
            w = cp.where(cross_region, w * 1.2, w)
            
            for j, wij in zip(cp.asnumpy(knn_idx), cp.asnumpy(w)):
                rows.append(i_global)
                cols.append(int(j))
                weights.append(float(wij))
    
    print(f"  k-NN computed in {time.time()-t0:.1f}s")
    
    rows = cp.array(rows, dtype=cp.int32)
    cols = cp.array(cols, dtype=cp.int32)
    weights = cp.array(weights, dtype=cp.float64)
    
    W = cp_coo((weights, (rows, cols)), shape=(N, N)).tocsr()
    W = 0.5 * (W + W.T)
    
    degrees = cp.array(W.sum(axis=1)).flatten()
    D = cp_diags(degrees)
    
    if normalized:
        # Normalized: L = I - D^{-1/2}WD^{-1/2}, eigenvalues in [0,2]
        degrees = cp.maximum(degrees, 1e-10)
        D_inv_sqrt = cp_diags(1.0 / cp.sqrt(degrees))
        I = cp_diags(cp.ones(N))
        L = I - D_inv_sqrt @ W @ D_inv_sqrt
    else:
        # Unnormalized: L = D - W, eigenvalues grow with N
        L = D - W
    
    L = cp_csr(L)
    print(f"  Laplacian ({'normalized' if normalized else 'unnormalized'}): {N}×{N}, nnz={L.nnz:,}")
    return L

print("="*60)
print("G₂ GRAPH LAPLACIAN (Unnormalized)")
print("="*60)

L_tcs = build_g2_laplacian(X_tcs, labels_tcs, k=80, normalized=False)

In [ ]:
# ============================================================
# CELL 5: Eigenvalue Computation
# ============================================================

def compute_spectrum(L: cp_csr, n_eigs: int = 50) -> np.ndarray:
    """Compute smallest eigenvalues of Laplacian."""
    N = L.shape[0]
    n_eigs = min(n_eigs, N - 2)
    
    print(f"  Computing {n_eigs} eigenvalues...")
    t0 = time.time()
    
    # Clear GPU memory
    cp.get_default_memory_pool().free_all_blocks()
    
    try:
        # CuPy eigsh with 'SA' (smallest algebraic)
        eigs, _ = cp_eigsh(L, k=n_eigs, which='SA')
        eigs = cp.sort(eigs)
        eigs = cp.asnumpy(eigs)
    except Exception as e:
        print(f"  GPU failed: {e}, trying CPU...")
        import scipy.sparse.linalg as sp_la
        L_cpu = L.get()
        eigs, _ = sp_la.eigsh(L_cpu, k=n_eigs, which='SA')
        eigs = np.sort(eigs)
    
    print(f"  Done in {time.time()-t0:.1f}s")
    
    # Filter out near-zero eigenvalue (constant mode)
    eigs = eigs[eigs > 1e-8]
    
    return eigs

# Compute spectrum
print("="*60)
print("SPECTRUM COMPUTATION")
print("="*60)

eigenvalues_raw = compute_spectrum(L_tcs, n_eigs=50)
print(f"\n  First 10 raw eigenvalues: {eigenvalues_raw[:10].round(6)}")

In [ ]:
# ============================================================
# CELL 6: Normalize and Compare to Riemann
# ============================================================

# Normalize: λ₁ × H* = γ₁
lambda1_target = RIEMANN_ZEROS[0] / H_STAR
scale_factor = lambda1_target / eigenvalues_raw[0]
eigenvalues = eigenvalues_raw * scale_factor

# Predicted gammas
n_compare = min(30, len(eigenvalues))
gamma_pred = eigenvalues[:n_compare] * H_STAR
gamma_actual = RIEMANN_ZEROS[:n_compare]

# Ratios
ratios_k7 = eigenvalues[:n_compare] / eigenvalues[0]
ratios_riemann = RIEMANN_RATIOS[:n_compare]
ratio_dev = np.abs(ratios_k7 - ratios_riemann) / ratios_riemann * 100

print("="*80)
print("SPECTRAL COMPARISON: K₇ (TCS) vs RIEMANN")
print("="*80)
print(f"\n  Scale factor: {scale_factor:.6f}")
print(f"  λ₁ = {eigenvalues[0]:.6f}, γ₁ = {gamma_pred[0]:.4f}")
print(f"\n{'n':>3} | {'λₙ':>10} | {'γₙ(K₇)':>10} | {'γₙ(Riem)':>10} | {'λₙ/λ₁':>8} | {'γₙ/γ₁':>8} | {'Δ%':>6}")
print("-"*78)

for i in range(n_compare):
    star = '★★★' if ratio_dev[i] < 1 else '★★' if ratio_dev[i] < 5 else '★' if ratio_dev[i] < 10 else ''
    print(f"{i+1:3} | {eigenvalues[i]:10.6f} | {gamma_pred[i]:10.4f} | "
          f"{gamma_actual[i]:10.4f} | {ratios_k7[i]:8.4f} | {ratios_riemann[i]:8.4f} | "
          f"{ratio_dev[i]:5.2f}% {star}")

# Summary
m1 = int(np.sum(ratio_dev < 1))
m5 = int(np.sum(ratio_dev < 5))
m10 = int(np.sum(ratio_dev < 10))

print(f"\n{'='*78}")
print(f"  Matches < 1%:  {m1}/{n_compare}")
print(f"  Matches < 5%:  {m5}/{n_compare}")
print(f"  Matches < 10%: {m10}/{n_compare}")
print(f"  Mean deviation: {np.mean(ratio_dev):.2f}%")

In [ ]:
# ============================================================
# CELL 7: Visualization
# ============================================================
try:
    import matplotlib.pyplot as plt
    HAS_PLT = True
except:
    HAS_PLT = False
    print("matplotlib not available")

if HAS_PLT:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Spectral ratios
    ax = axes[0, 0]
    n_plot = len(ratios_k7)
    ax.plot(range(1, n_plot+1), ratios_riemann, 'ko-', lw=2, ms=6, label='Riemann γₙ/γ₁')
    ax.plot(range(1, n_plot+1), ratios_k7, 'rs--', lw=2, ms=5, alpha=0.7, label='K₇ (TCS) λₙ/λ₁')
    ax.set_xlabel('Index n')
    ax.set_ylabel('Spectral Ratio')
    ax.set_title('Spectral Ratios: K₇ TCS vs Riemann')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. γₙ scatter
    ax = axes[0, 1]
    sc = ax.scatter(gamma_actual, gamma_pred, c=ratio_dev, cmap='RdYlGn_r', 
                    s=80, edgecolors='k')
    ax.plot([10, 110], [10, 110], 'k--', lw=2)
    ax.set_xlabel('γₙ (Riemann)')
    ax.set_ylabel('γₙ (K₇ TCS)')
    ax.set_title('γₙ Correspondence')
    plt.colorbar(sc, ax=ax, label='Deviation %')
    
    # 3. Deviation by index
    ax = axes[1, 0]
    colors = ['green' if d < 5 else 'orange' if d < 10 else 'red' for d in ratio_dev]
    ax.bar(range(1, n_plot+1), ratio_dev, color=colors, edgecolor='k')
    ax.axhline(5, color='orange', ls='--', lw=2, label='5%')
    ax.axhline(10, color='red', ls='--', lw=2, label='10%')
    ax.set_xlabel('Index n')
    ax.set_ylabel('Deviation %')
    ax.set_title('Ratio Deviations')
    ax.legend()
    
    # 4. TCS structure visualization (x₇ histogram)
    ax = axes[1, 1]
    x7_cpu = cp.asnumpy(X_tcs[:, 6])
    labels_cpu = cp.asnumpy(labels_tcs)
    ax.hist(x7_cpu[labels_cpu == 0], bins=30, alpha=0.6, label='Z₊', color='blue')
    ax.hist(x7_cpu[labels_cpu == 1], bins=30, alpha=0.6, label='Neck', color='green')
    ax.hist(x7_cpu[labels_cpu == 2], bins=30, alpha=0.6, label='Z₋', color='red')
    ax.set_xlabel('x₇ (gluing direction)')
    ax.set_ylabel('Point count')
    ax.set_title('TCS Structure')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('K7_Riemann_v4_TCS_results.png', dpi=150)
    plt.show()
    print("\\n✓ Saved: K7_Riemann_v4_TCS_results.png")

In [ ]:
# ============================================================
# CELL 8: Final Summary & Export
# ============================================================
import json

print("="*80)
print("█" + " "*30 + "FINAL RESULTS" + " "*31 + "█")
print("="*80)

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│  K₇ RIEMANN VERIFICATION v4 — TWISTED CONNECTED SUM                          │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  TOPOLOGY: K₇ = (Z₊ × S¹) ∪_φ (Z₋ × S¹)                                      │
│    b₂ = {B2_K7}, b₃ = {B3_K7}, H* = {H_STAR}                                              │
│                                                                              │
│  DISCRETIZATION: {len(X_tcs):,} points (Z₊: 5000, Neck: 2000, Z₋: 5000)          │
│                                                                              │
│  SPECTRAL RATIO RESULTS:                                                     │
│    Matches < 1%:  {m1:2d}/{n_compare}                                                     │
│    Matches < 5%:  {m5:2d}/{n_compare}                                                     │
│    Matches < 10%: {m10:2d}/{n_compare}                                                     │
│    Mean deviation: {np.mean(ratio_dev):5.2f}%                                             │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

# Export
results = {
    'version': '4.0',
    'method': 'TCS Graph Laplacian',
    'topology': {'b2': B2_K7, 'b3': B3_K7, 'H_star': H_STAR},
    'discretization': {
        'N_total': int(len(X_tcs)),
        'N_bulk': 5000,
        'N_neck': 2000
    },
    'eigenvalues': eigenvalues[:n_compare].tolist(),
    'gamma_predicted': gamma_pred.tolist(),
    'gamma_actual': gamma_actual.tolist(),
    'ratio_deviations': ratio_dev.tolist(),
    'summary': {
        'matches_1pct': m1,
        'matches_5pct': m5,
        'matches_10pct': m10,
        'mean_deviation': float(np.mean(ratio_dev))
    }
}

with open('K7_Riemann_v4_TCS_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Saved: K7_Riemann_v4_TCS_results.json")

In [ ]:
# ============================================================
# CELL 9: Test Alternative Hypotheses
# ============================================================
# L'hypothèse linéaire γₙ = λₙ × H* échoue à cause de Weyl.
# Testons des relations alternatives!

import numpy as np

# Données actuelles
lambda_n = np.array(eigenvalues[:30])
gamma_actual = np.array(RIEMANN_ZEROS[:30])

print("="*70)
print("TEST D'HYPOTHÈSES ALTERNATIVES")
print("="*70)

# Hypothesis 1: γₙ = exp(a × λₙ + b)
# => log(γₙ) = a × λₙ + b
log_gamma = np.log(gamma_actual)
# Fit linéaire
a1, b1 = np.polyfit(lambda_n, log_gamma, 1)
gamma_pred_exp = np.exp(a1 * lambda_n + b1)
dev_exp = np.abs(gamma_pred_exp - gamma_actual) / gamma_actual * 100

print(f"\n[H1] log(γₙ) = a×λₙ + b  =>  γₙ = exp(a×λₙ + b)")
print(f"     Fit: a = {a1:.4f}, b = {b1:.4f}")
print(f"     Mean deviation: {np.mean(dev_exp):.2f}%")
print(f"     Max deviation: {np.max(dev_exp):.2f}%")

# Hypothesis 2: γₙ = a × λₙ^c (power law)
# => log(γₙ) = log(a) + c × log(λₙ)
log_lambda = np.log(lambda_n)
c2, log_a2 = np.polyfit(log_lambda, log_gamma, 1)
a2 = np.exp(log_a2)
gamma_pred_pow = a2 * lambda_n ** c2
dev_pow = np.abs(gamma_pred_pow - gamma_actual) / gamma_actual * 100

print(f"\n[H2] γₙ = a × λₙ^c  (power law)")
print(f"     Fit: a = {a2:.4f}, c = {c2:.4f}")
print(f"     Mean deviation: {np.mean(dev_pow):.2f}%")
print(f"     Max deviation: {np.max(dev_pow):.2f}%")

# Hypothesis 3: γₙ = a × n^b (index-based, ignoring λₙ)
n_vals = np.arange(1, 31)
b3, log_a3 = np.polyfit(np.log(n_vals), log_gamma, 1)
a3 = np.exp(log_a3)
gamma_pred_idx = a3 * n_vals ** b3
dev_idx = np.abs(gamma_pred_idx - gamma_actual) / gamma_actual * 100

print(f"\n[H3] γₙ = a × n^b  (index-based)")
print(f"     Fit: a = {a3:.4f}, b = {b3:.4f}")
print(f"     Mean deviation: {np.mean(dev_idx):.2f}%")
print(f"     Max deviation: {np.max(dev_idx):.2f}%")

# Hypothesis 4: Spectral index mapping
# Maybe λ values map to DIFFERENT indices in Riemann sequence?
# Check if eigenvalue ORDERING matches any Riemann subsequence
print(f"\n[H4] Check eigenvalue spacing pattern")
lambda_gaps = np.diff(lambda_n)
gamma_gaps = np.diff(gamma_actual)
gap_ratio = gamma_gaps / (lambda_gaps * H_STAR)
print(f"     γ gaps / (λ gaps × H*): min={gap_ratio.min():.2f}, max={gap_ratio.max():.2f}")
print(f"     If constant, linear hypothesis holds. Varies = nonlinear relationship.")

# Summary
print(f"\n{'='*70}")
print("RÉSUMÉ: Meilleure hypothèse?")
print(f"{'='*70}")
results = [
    ("H1: Exponentielle", np.mean(dev_exp)),
    ("H2: Power law", np.mean(dev_pow)),
    ("H3: Index-based", np.mean(dev_idx)),
]
results.sort(key=lambda x: x[1])
for name, dev in results:
    print(f"  {name}: {dev:.2f}% mean deviation")

In [ ]:
# ============================================================
# CELL 10: Deep Dive - Index Mapping Analysis
# ============================================================
# Si γₙ = a × n^b fonctionne, alors K₇ encode peut-être n via λₙ!
# Cherchons la transformation: λₙ → n → γₙ

print("="*70)
print("ANALYSE APPROFONDIE: MAPPING λₙ → n → γₙ")
print("="*70)

# 1. Fit Riemann zeros to power law: γₙ = a × n^b
n_vals = np.arange(1, 31)
log_n = np.log(n_vals)
log_gamma = np.log(gamma_actual)

b_riemann, log_a_riemann = np.polyfit(log_n, log_gamma, 1)
a_riemann = np.exp(log_a_riemann)

print(f"\n[1] Fit Riemann: γₙ = {a_riemann:.4f} × n^{b_riemann:.4f}")

gamma_from_n = a_riemann * n_vals ** b_riemann
riemann_fit_error = np.mean(np.abs(gamma_from_n - gamma_actual) / gamma_actual * 100)
print(f"    Erreur moyenne du fit: {riemann_fit_error:.2f}%")

# 2. Inverse: given γₙ, solve for n
# n = (γₙ / a)^(1/b)
def gamma_to_index(gamma, a=a_riemann, b=b_riemann):
    return (gamma / a) ** (1/b)

# 3. Now: can we find a mapping λₙ → n?
# Try: n = c × (λₙ - λ₁)^d + 1  (shift and power)
lambda_shifted = lambda_n - lambda_n[0]  # shift so λ₁ → 0
lambda_shifted[0] = 1e-10  # avoid log(0)

# Fit: log(n-1) vs log(λₙ - λ₁) for n > 1
valid_idx = lambda_shifted > 1e-8
if np.sum(valid_idx) > 5:
    log_lambda_shift = np.log(lambda_shifted[valid_idx])
    log_n_shift = np.log(n_vals[valid_idx])
    
    d_fit, log_c_fit = np.polyfit(log_lambda_shift, log_n_shift, 1)
    c_fit = np.exp(log_c_fit)
    
    print(f"\n[2] Mapping λₙ → n: n ≈ {c_fit:.2f} × (λₙ - λ₁)^{d_fit:.4f}")
    
    # Predict n from λ
    n_predicted = c_fit * (lambda_shifted ** d_fit)
    n_predicted[0] = 1  # fix first
    
    # Then predict γ from n
    gamma_predicted_new = a_riemann * n_predicted ** b_riemann
    
    new_error = np.abs(gamma_predicted_new - gamma_actual) / gamma_actual * 100
    print(f"    Erreur γ avec ce mapping: {np.mean(new_error):.2f}%")

# 4. Alternative: Linear mapping n = α × λₙ + β
alpha, beta = np.polyfit(lambda_n, n_vals, 1)
n_from_lambda_linear = alpha * lambda_n + beta
gamma_from_linear = a_riemann * np.maximum(n_from_lambda_linear, 1) ** b_riemann
linear_error = np.abs(gamma_from_linear - gamma_actual) / gamma_actual * 100

print(f"\n[3] Linear: n = {alpha:.2f} × λₙ + {beta:.2f}")
print(f"    Erreur γ: {np.mean(linear_error):.2f}%")

# 5. Check if GIFT constants appear
print(f"\n[4] Recherche de constantes GIFT:")
print(f"    H* = {H_STAR}")
print(f"    dim(G₂) = {DIM_G2}")
print(f"    b₂ = {B2_K7}, b₃ = {B3_K7}")
print(f"    a_riemann = {a_riemann:.4f} ≈ γ₁ = {gamma_actual[0]:.4f}? {np.isclose(a_riemann, gamma_actual[0], rtol=0.1)}")
print(f"    b_riemann = {b_riemann:.4f} ≈ 2/7 = {2/7:.4f}? (Weyl) {np.isclose(b_riemann, 2/7, rtol=0.2)}")
print(f"    b_riemann ≈ dim(G₂)/H* = {DIM_G2/H_STAR:.4f}? {np.isclose(b_riemann, DIM_G2/H_STAR, rtol=0.5)}")

# 6. Ultimate test: use GIFT constants directly
print(f"\n[5] Test avec constantes GIFT:")
# Hypothesis: γₙ = γ₁ × n^(b₂/H*)  or similar
for exp_test in [B2_K7/H_STAR, DIM_G2/H_STAR, 2/7, B3_K7/H_STAR, 0.5, 0.6, 0.7]:
    gamma_test = gamma_actual[0] * n_vals ** exp_test
    err_test = np.mean(np.abs(gamma_test - gamma_actual) / gamma_actual * 100)
    print(f"    γₙ = γ₁ × n^{exp_test:.4f}: erreur = {err_test:.2f}%")